In [143]:
import pandas as pd
import numpy as np
import random

In [2]:
population = pd.read_csv('./WorldData/PopulationWorldbank.csv', skiprows=4, usecols=['Country Name', 'Country Code', '2020'])
populationBackup = population.copy(deep=True)
populationTotal = population.loc[population['Country Name'] == 'World', '2020'].values[0]

missingCountries = {'Eritrea' : ('ERI', 3.213 * 1e6), 'Saint Barthélemy' : ('BLM', 9131), 'Saint Helena' : ('SHN', 4255),
                    'Saint Pierre and Miquelon' : ('SPM', 5888), 'Taiwan' : ('TWN', 23.57 * 1e6), 'Tokelau' : ('TKL', 1411),
                    'Wallis and Futuna Islands' : ('WLF', 15289), 'Holy See' : ('VAT', 825), 'Niue' : ('NIU', 1620),
                    'Bonaire, Sint Eustatius and Saba' : ('BES', 26000), 'Cook Islands': ('COK', 17459)}

"""
'Anguilla' : 15094, ,
                    , 'Falkland Islands (Malvinas)' : 2840, 'French Guiana' : 294071,
                    'Guadeloupe' : 395700, , 'Martinique' : 376480, 'Mayotte' :  270372,
                    'Montserrat' : 4992, , 'Réunion' : 859959, 
"""

excludedCodes = ['WLD', 'IBT', 'LMY', 'MIC', 'IBRD', 'IBD', 'EAR', 'LMC', 'UMC', 'EAS', 'LTE', 'EAP', 'TEA', 'SAS', 'TSA',
                'IDA', 'OED', 'HIC', 'SSF', 'TSS', 'SSA', 'IDX', 'PST', 'PRE', 'LDC', 'FCS', 'ECS', 'HPC', 'AFE', 'LIC',
                'LCN', 'TLA', 'LAC', 'MEA', 'IDB', 'TEC', 'AFW', 'EUU', 'ARB', 'ECA', 'MNA', 'TMN', 'NAC', 'EMU', 'OSS',
                'CEB', 'SST', 'CSS', 'PSS', 'INX', 'ERI']

df_missingCountries = {'Country Name' : [], 'Country Code' : [], '2020' : []}
for k,v in missingCountries.items():
    df_missingCountries['Country Name'].append(k)
    df_missingCountries['Country Code'].append(v[0])
    df_missingCountries['2020'].append(v[1])
    
population = population.append(pd.DataFrame(df_missingCountries), ignore_index=True)
    
population['Country Code'] = population['Country Code'].replace(np.nan, '---')
population = population[~population['Country Code'].isin(excludedCodes)]
population['World%'] = population['2020'] / populationTotal
totalPercentage = np.sum(population['World%'])
population.sort_values(by=['World%'], ascending=False, inplace=True)

if not abs(totalPercentage - 1) < 1e-3:
    print(populationBackup[populationBackup['Country Code'].isin(excludedCodes)][['Country Name', 'Country Code']])
    print(population.head(15))
    print(f'Number of entities: {len(population)}')
    population.to_excel('PopulationWrong.xlsx')
    if totalPercentage < 1:
        raise AssertionError(f'List incomplete, total population percentage: {totalPercentage}')
    else:
        raise AssertionError(f'List contains additional entities, total population percentage: {totalPercentage}')
    
populationTotal *= totalPercentage
population['World%'] = population['2020'] / populationTotal
population

,Country Name,Country Code,2020,World%
40,China,CHN,1.402112e+09,1.809326e-01
109,India,IND,1.380004e+09,1.780797e-01
251,United States,USA,3.294841e+08,4.251758e-02
106,Indonesia,IDN,2.735236e+08,3.529628e-02
184,Pakistan,PAK,2.208923e+08,2.850458e-02
...,...,...,...,...
269,Saint Pierre and Miquelon,SPM,5.888000e+03,7.598045e-07
268,Saint Helena,SHN,4.255000e+03,5.490774e-07
274,Niue,NIU,1.620000e+03,2.090495e-07
271,Tokelau,TKL,1.411000e+03,1.820795e-07


In [124]:
probs = population['World%'].values
countries = population['Country Name'].values

def random_countries(count):
    random_countries = np.random.choice(countries, count, p=probs)

    for i in random_countries:
        your_country = i
        row = population.loc[population['Country Name'] == your_country]
        code = row['Country Code'].values[0]
        probability = row['World%'].values[0]
        padding = (26 - len(your_country)) * ' '
        death = random_death(code)
        print(f'Your lucky roll is: {code}, {your_country}{padding}{round(probability * 10000) / 100} %\n       Death cause: {death}\n')

In [140]:
scrapedNames2code = pd.read_csv('./WebScraped/ScrapedNames2Codes.csv').set_index('Code')
def random_death(countryCode):
    try:
        death_causes = pd.read_csv('./WebScraped/DeathCauses/'+scrapedNames2code.loc[countryCode]['Country']+'.csv').drop('Unnamed: 0', axis=1).set_index('DeathCause')
    except:
        return 'Missing data'
    death_causes['Percentage'] /= 100
    death_causes.loc['Other'] = [1 - np.sum(death_causes['Percentage'])]
    random_death_cause = np.random.choice(death_causes.index.values, 1, p=death_causes['Percentage'].values)[0]
    padding = (31 - len(random_death_cause)) * ' '
    return (random_death_cause + padding + str(round(death_causes.loc[random_death_cause].values[0] * 10000) / 100) + ' %')

In [145]:
random_countries(1)

Your lucky roll is: NPL, Nepal                     0.38 %
       Death cause: Tuberculosis                   3.9 %

